In [1]:
%%time
import boto3
import json
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model


CPU times: user 649 ms, sys: 146 ms, total: 795 ms
Wall time: 742 ms


In [2]:
sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = "mysagemakertutorial"
data_prefix = "sagemaker/data"  
model_prefix = "sagemaker/model"
role = get_execution_role()

In [3]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")
xgboost_container = sagemaker.image_uris.retrieve("xgboost", region, "latest")
xgboost_container

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest'

In [4]:
model_name = 'xgboost-2021-12-22-16-10-49-300'
model_url = 'https://mysagemakertutorial.s3.us-west-2.amazonaws.com/sagemaker/model/xgboost-2021-12-22-16-10-49-300/output/model.tar.gz'

In [5]:
trainedmodel = sagemaker.model.Model(
    model_data=model_url, 
    image_uri = xgboost_container,
#     '246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost',
    role=role)  # your role here; could be different name

In [6]:
# The location of the test dataset
batch_input = 's3://{}/{}/test'.format(bucket, data_prefix)

# The location to store the results of the batch transform job
batch_output = 's3://{}/{}/batch-prediction'.format(bucket, model_prefix)

In [7]:
transformer = trainedmodel.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge', 
    output_path=batch_output
)

In [8]:
%%time
transformer.transform(
    data=batch_input, 
    data_type='S3Prefix',
    content_type='text/csv', 
    split_type='Line'
)

................................Arguments: serve
[2021-12-22 17:00:02 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2021-12-22 17:00:02 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2021-12-22 17:00:02 +0000] [1] [INFO] Using worker: gevent
[2021-12-22 17:00:02 +0000] [21] [INFO] Booting worker with pid: 21
[2021-12-22 17:00:02 +0000] [22] [INFO] Booting worker with pid: 22
[2021-12-22 17:00:02 +0000] [23] [INFO] Booting worker with pid: 23
[2021-12-22 17:00:02 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py

In [67]:
! aws s3 cp {batch_output} ./ --recursive


download: s3://mysagemakertutorial/sagemaker/model/batchprediction/test.csv.out to ./test.csv.out
